### Psycopg2

1. Wczytaj dane z pliku `cars.csv` i wyodrębnij z niego kolumny:
   - `price`
   - `brand`
   - `fuel`
   - `power`
   - `prod_year`


Następnie napisz zapytanie `CREATE`, które utworzy tabelę o takim samym schemacie + dodatkowa kolumna `id` typu `serial`. Wykonaj zapytanie za pomocą `psycopg2`

---
(czas: 16 min.)

In [8]:
# ...

2. Napisz kod, który na podstawie numeru indeksu wiersza dataframe'a, wyciągnie ten wiersz z tabeli, utworzy zapytanie `INSERT` a następnie wykona je na bazie.

Podpowiedź:

Poniższy kod zamieni typy danych w rekordzie z df-a z numpyowych na zwykłe. Będzie to przydatne podczas insertowania.

`[x.item() if isinstance(x, np.generic) else x for x in record.values]`

---

(czas: 20 min.)

In [1]:
# ...

3. Napisz funkcję, która przyjmie zapytanie typu `SELECT` i wykona je na bazie.

In [7]:
# ...

4. Napisz funkcję, która przyjmie `id` wiersza w tabeli, nazwę kolumny oraz nową wartość a następnie zupdatuje tabelę.

In [5]:
# ...

5. Napisz funkcję, która przyjmie `id` wiersza w tabeli a następnie go usunie.

In [6]:
# ...

### SQLAlchemy

1. Utwórz klasę odpowiadającą tabeli `cars` a następnie samą tabelę za pomocą `SQLAlchemy`.


---
(czas: 16 min.)

In [2]:
# ...

2. Napisz kod, który doda do tabeli wiersz z dataframe'a wykorzystując `SQLAlchemy`.

   
---
(czas: 8 min.)

In [3]:
# ...

3. Napisz kod, który wyciągnie dane z dataframe'a na różne sposoby

In [4]:
# ...